# Assignment 2.1 - Data Lake

## Dependency Setup

In [2]:
!python --version

Python 3.11.11


In [3]:
# Combining all installations into one cell
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null
!pip install --disable-pip-version-check -q awscli boto3
!pip install --disable-pip-version-check -q sagemaker
!pip install --disable-pip-version-check -q smdebug
!pip install --disable-pip-version-check -q sagemaker-experiments
!pip install --disable-pip-version-check -q PyAthena
!pip install --disable-pip-version-check -q awswrangler
!pip install --disable-pip-version-check -q matplotlib
!pip install --disable-pip-version-check -q seaborn

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
aiobotocore 2.13.3 requires botocore<1.34.163,>=1.34.70, but you have botocore 1.36.2 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-core 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which is incompatible.
autogluon-core 1.1.1 requires scipy<1.13,>=1.5.4, but you have scipy 1.14.1 which is incompatible.
autogluon-features 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which is incompatible.
autogluon-multimodal 1.1.1 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omega

In [8]:
!pip list

Package                                 Version
--------------------------------------- --------------
absl-py                                 2.1.0
accelerate                              0.21.0
aiobotocore                             2.13.3
aiohttp                                 3.9.5
aiohttp-cors                            0.7.0
aioitertools                            0.12.0
aiosignal                               1.3.1
aiosqlite                               0.19.0
alembic                                 1.14.0
altair                                  5.4.1
amazon-q-developer-jupyterlab-ext       3.4.5
amazon_sagemaker_jupyter_ai_q_developer 1.0.14
amazon_sagemaker_jupyter_scheduler      3.1.8
amazon-sagemaker-sql-editor             0.1.14
amazon-sagemaker-sql-execution          0.1.6
amazon-sagemaker-sql-magic              0.1.3
annotated-types                         0.7.0
ansi2html                               1.9.2
ansicolors                              1.1.8
antlr4-python3-r

In [4]:
!conda install -y zip

Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - zip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    amazon-sagemaker-jupyter-scheduler-3.1.8|     pyhd8ed1ab_0         169 KB  conda-forge
    amazon_sagemaker_sql_editor-0.1.14|     pyhd8ed1ab_0        17.2 MB  conda-forge
    aws-glue-sessions-1.0.8    |     pyhd8ed1ab_0          34 KB  conda-forge
    ca-certificates-2024.12.14 |       hbcca054_0         153 KB  conda-forge
    certifi-2024.12.14         |     pyhd8ed1ab_0         158 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    jinja2-3.1.5               |     pyhd8ed1ab_0         110 KB  conda-forge
    jupyter-ai-2.28.5          |     pyhd8ed1ab_0         742 KB  conda-forge
    jupyter-ai-magics-2.28.5   |     pyhd8ed1ab_

### Dependency Summary

In [9]:
setup_dependencies_passed = True

In [10]:
%store setup_dependencies_passed

Stored 'setup_dependencies_passed' (bool)


In [11]:
%store

Stored variables and their in-db values:
setup_dependencies_passed             -> True


### Dependency Imports

In [14]:
# AWS Imports
import boto3
from botocore.client import ClientError
import sagemaker
from pyathena import connect
import awswrangler as wr

# Data Transformation Imports
import pandas as pd

# Misc Imports
from IPython.display import display, HTML

## S3 Initialization

In [15]:
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [16]:
setup_s3_bucket_passed = False

In [17]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-203012117619


### Bucket Verification

In [18]:
response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': 'P053QN5GVEDPDC13', 'HostId': 'dAlPiYT56MYYbDrrFzBkx0ppD4meI2oCKKOJEcwpU+8UucwbqIXAE1e3lSrsruWGcrUFFmPRRAA=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'dAlPiYT56MYYbDrrFzBkx0ppD4meI2oCKKOJEcwpU+8UucwbqIXAE1e3lSrsruWGcrUFFmPRRAA=', 'x-amz-request-id': 'P053QN5GVEDPDC13', 'date': 'Sun, 19 Jan 2025 19:36:34 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [19]:
%store setup_s3_bucket_passed

Stored 'setup_s3_bucket_passed' (bool)


In [20]:
%store

Stored variables and their in-db values:
setup_dependencies_passed             -> True
setup_s3_bucket_passed                -> True


## Copy Dataset to S3 Bucket

In [26]:
try:
    setup_dependencies_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++")

In [27]:
print(setup_dependencies_passed)

True


In [28]:
%store -r setup_s3_bucket_passed

In [29]:
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")

In [30]:
print(setup_s3_bucket_passed)

True


In [31]:
if not setup_dependencies_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
if not setup_s3_bucket_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [32]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

### Setting S3 Destination Location

In [33]:
s3_private_path_csv = "s3://{}/assignment-2/csv".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-203012117619/assignment-2/csv


In [34]:
%store s3_private_path_csv

Stored 's3_private_path_csv' (str)


### Copying Data from Local Directory to S3

In [35]:
!aws s3 cp "dataset.csv" $s3_private_path_csv/

upload: ./dataset.csv to s3://sagemaker-us-east-1-203012117619/assignment-2/csv/dataset.csv


### Listing Files in our Bucket

In [36]:
print(s3_private_path_csv)

s3://sagemaker-us-east-1-203012117619/assignment-2/csv


In [37]:
!aws s3 ls $s3_private_path_csv/

2025-01-19 19:48:31    1048576 dataset.csv


In [38]:
display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-{}-{}/assignment-2/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

## Creating Athena Schema

In [41]:
ingest_create_athena_db_passed = False

In [40]:
%store -r s3_public_path_csv

no stored variable or alias s3_public_path_csv


In [42]:
try:
    s3_public_path_csv
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

*****************************************************************************
[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************
[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************
*****************************************************************************


## Registering Data with Athena